In [1]:
import pandas as pd
import utils as U
import requests
import awswrangler as wr
import importlib
importlib.reload(wr)

<module 'awswrangler' from '/home/jyotiraditya/anaconda3/envs/SiteReports/lib/python3.10/site-packages/awswrangler/__init__.py'>

In [2]:
# def create_project(lat,lng,name,travel_mode,cost_type,cost):
#     url=f"http://127.0.0.1:8000/site_report/create/?lat={lat}&lng={lng}&name={name}&travel_mode={travel_mode}&cost_type={cost_type}&cost={cost}"
#     r = requests.get(url)
#     print(r.text)
# 
# locations=[(12.934247287987455, 77.7438690876081, 'Orion Utopia'),
#               (12.805019480136755, 77.69968439147448, 'Orion Valencia')]
# for location in locations:
#     lat,lng,name=location
#     for cf in [('driving','time','15'),('driving','distance','500')]:
#         travel_mode,cost_type,cost=cf
#         create_project(lat=lat,lng=lng,name=name,travel_mode=travel_mode,cost_type=cost_type,cost=cost)



{"project_id":{"report_id":"f9b43dc5-f8fe-4cb0-b91a-54cbf23de536","id":"129342_777438"}}
{"project_id":{"report_id":"ba9999e5-b9f2-4eb0-8b71-eec0c9af1930","id":"129342_777438"}}
{"project_id":{"report_id":"59237f70-29b4-463c-a869-8323df9f54ca","id":"128050_776996"}}
{"project_id":{"report_id":"df93c3fa-d9ac-46ad-9b8a-d423fb715ae6","id":"128050_776996"}}


In [3]:
df = pd.DataFrame(columns=['a', 'b'])
df.groupby(['a', 'b'])

In [3]:
all_reports = U.get_all_reports().query("catchment_type=='i500mtb'")


In [6]:
query = '''UPDATE ind_site_reports_v2
SET
    geometry=:geometry,
    catchment_type=:catchment_type
WHERE  id=:id and report_id=:report_id;
'''
con, cur = U.connect_to_db()
for report in all_reports.to_dict('records'):
    id = report['id']
    report_id = report['report_id']
    lat = report['lat']
    lng = report['lng']
    travel_mode = 'driving'
    cost_type = 'distance'
    cost = 500
    isochrone_polygon, isochrone_id = U.get_isochrone(lat, lng, travel_mode=travel_mode, cost_type=cost_type,
                                                      cost=cost)
    query_params = dict(geometry=isochrone_polygon.wkt, catchment_type=isochrone_id, id=id, report_id=report_id)
    cur.execute(query, query_params)
    con.commit()
con.close()


13.011026921074786 77.55473637957641 driving distance 500 pk.eyJ1IjoibGFpcjA4MjYiLCJhIjoiY2tkcGoxcnRzMDZvODJxbXk0MWhlcWN2aSJ9.5-yjt_SUq4w5JII7CvD4cA
13.0009569 77.6325712 driving distance 500 pk.eyJ1IjoibGFpcjA4MjYiLCJhIjoiY2tkcGoxcnRzMDZvODJxbXk0MWhlcWN2aSJ9.5-yjt_SUq4w5JII7CvD4cA
13.0552603 77.7638939 driving distance 500 pk.eyJ1IjoibGFpcjA4MjYiLCJhIjoiY2tkcGoxcnRzMDZvODJxbXk0MWhlcWN2aSJ9.5-yjt_SUq4w5JII7CvD4cA
13.06317 77.620569 driving distance 500 pk.eyJ1IjoibGFpcjA4MjYiLCJhIjoiY2tkcGoxcnRzMDZvODJxbXk0MWhlcWN2aSJ9.5-yjt_SUq4w5JII7CvD4cA
12.887373 77.596901 driving distance 500 pk.eyJ1IjoibGFpcjA4MjYiLCJhIjoiY2tkcGoxcnRzMDZvODJxbXk0MWhlcWN2aSJ9.5-yjt_SUq4w5JII7CvD4cA


In [2]:
site_reports = wr.athena.read_sql_query("select * from datasets_prep.ind_site_reports_data_v2",
                                        database='datasets_prep')

/home/jyotiraditya/anaconda3/envs/SiteReports/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by promote_options='default'.
  return cls.dispatch_func(func)(*args, **kw)


In [3]:
all_reports = site_reports.copy()
mask = all_reports['catchment_type'].isin(('i15mind', 'i2500mtd'))
all_reports[mask].shape

(9, 29)

In [15]:
all_reports.columns

Index(['report_id', 'id', 'site_name', 'lat', 'lng', 'geometry', 'location',
       'catchment_type', 'top_brands', 'pois', 'projects', 'apartments',
       'household_distribution', 'competition', 'population', 'affluence',
       'avg_cost_for_two', 'revenue_score', 'high_streets', 'shopping_malls',
       'created_at', 'location_score_weights', 'grouped_indexes',
       'demand_generators', 'city_lat', 'city_lng',
       'distance_from_city_center', 'orientation_from_city_center',
       'competitors_domains'],
      dtype='object')

In [4]:
import json
import re
import revenue_score as rev
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=5,progress_bar=True)

INFO: Pandarallel will run on 5 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
def get_catchment_info(row):
    pattern = re.compile("i(?P<cost>\d+)(?P<cost_type>[a-z]+)(?P<travel_mode>[d|b])$")
    catchment_type = row['catchment_type']
    a = pattern.search(catchment_type).groupdict()
    cost_types = {"mt": 'distance', 'min': "time"}
    travel_modes = {"d": 'driving', 'b': "buffer"}
    cost_type = cost_types[a['cost_type']]
    travel_mode = travel_modes[a['travel_mode']]
    cost = int(a['cost'])
    return travel_mode, cost_type, cost, catchment_type


def get_competitors(row):
    competitors = json.loads(row['competitors_domains'])
    primary_comp = list(filter(lambda x: x['competitor_type'] == 'primary_competitor', competitors))
    anchor_comp = list(filter(lambda x: x['competitor_type'] == 'anchor_competitor', competitors))
    if primary_comp:
        primary_comp = primary_comp[0]['filter']
        primary_comp = list(filter(lambda x: "'" not in x, primary_comp))
    if anchor_comp:
        anchor_comp = anchor_comp[0]['filter']
        anchor_comp = list(filter(lambda x: "'" not in x, anchor_comp))
    return primary_comp, anchor_comp


def get_revenue_score_and_grouped_indexes(row):
    if row['catchment_type'] not in ('i15mind','i2500mtd'):
        return row['revenue_score'],row['affluence'],row['grouped_indexes']
    weights_score = json.loads(row['location_score_weights'])
    lat, lng = row['lat'], row['lng']
    travel_mode, cost_type, cost, catchment_type = get_catchment_info(row)
    primary_comp, anchor_comp = get_competitors(row)
    revenue_score, grouped_indexs = rev.generate_revenue_score(lat, lng, travel_mode, cost_type, cost, catchment_type,
                                                               primary_comp,
                                                               anchor_comp, weights_score)
    affluence_score=grouped_indexs['affluence_index']
    return revenue_score, affluence_score,json.dumps(grouped_indexs)



In [7]:
all_reports[['revenue_score','affluence','grouped_indexes']]=all_reports.parallel_apply(get_revenue_score_and_grouped_indexes,axis=1,result_type='expand')

/media/jyotiraditya/Ultra Touch/repos/SiteReports/revenue_score.py:743: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  
/media/jyotiraditya/Ultra Touch/repos/SiteReports/revenue_score.py:743: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  
/media/jyotiraditya/Ultra Touch/repos/SiteReports/revenue_score.py:743: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining t

In [10]:
all_reports[['id','catchment_type','revenue_score','affluence','grouped_indexes','catchment_type']]

,id,catchment_type,revenue_score,affluence,grouped_indexes,catchment_type
0,129342_777438,i15mind,70.547094,3.11,"{""affluence_index"": 3.11, ""apartments_index"": ...",i15mind
1,128050_776996,i500mtd,65.019439,2.67,"{""affluence_index"": 2.67, ""apartments_index"": ...",i500mtd
2,129342_777438,i500mtd,71.910621,3.39,"{""affluence_index"": 3.39, ""apartments_index"": ...",i500mtd
3,130110_775547,i500mtd,71.938076,4.83,"{""affluence_index"": 4.83, ""apartments_index"": ...",i500mtd
4,130009_776325,i500mtd,63.958918,4.15,"{""affluence_index"": 4.15, ""apartments_index"": ...",i500mtd
5,130552_777638,i500mtd,49.996000,3.13,"{""affluence_index"": 3.13, ""apartments_index"": ...",i500mtd
6,130631_776205,i500mtd,81.386000,3.76,"{""affluence_index"": 3.76, ""apartments_index"": ...",i500mtd
7,130552_777638,i15mind,49.760922,1.59,"{""affluence_index"": 1.59, ""apartments_index"": ...",i15mind
8,128050_776996,i15mind,65.147295,2.56,"{""affluence_index"": 2.56, ""apartments_index"": ...",i15mind
9,130110_775547,i15mind,89.403407,3.64,"{""affluence_index"": 3.64, ""apartments_index"": ...",i15mind


In [29]:
all_reports.to_parquet("site_reports.parquet",index=False)

In [2]:
all_reports=pd.read_parquet("site_reports.parquet")

In [4]:
all_reports.shape

(18, 29)

In [11]:
wr.s3.to_parquet(all_reports, path="s3://tuzomldev/site_reports/", table="ind_site_reports_data_v2", database='datasets_prep',
                 dataset=True, mode='overwrite',
                 dtype={'report_id': 'string', 'id': 'string', 'site_name': 'string', 'lat': 'double',
                        'lng': 'double', 'catchment_type': 'string', 'top_brands': 'string', 'pois': 'string',
                        'projects': 'string', 'apartments': 'string', 'median_price': 'string',
                        'household_distribution': 'string', 'competition': 'string', 'population': 'string',
                        'affluence': 'double', 'avg_cost_for_two': 'double', 'revenue_score': 'double',
                        'high_streets': 'string', 'shopping_malls': 'string', 'poi_counts': 'string',
                        'projects_counts': 'string', 'created_at': 'bigint', "companies": "string"})

{'paths': ['s3://tuzomldev/site_reports/724537ad52654d5d93e9e79056e6b7d8.snappy.parquet'],
 'partitions_values': {}}